In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import qutip as qt
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
import qiskit
from itertools import product
from typing import List, Dict, Tuple

In [3]:
def create_qubo_matrix(G: np.ndarray | List, 
                       P: np.ndarray | List, 
                       V: np.ndarray, 
                       D: np.ndarray, 
                       lambda1: float | int = 10, 
                       lambda2: float | int =10) -> Dict:
    """
    Constructs the QUBO matrix for the gate assignment problem.

    Ref: H. Mohammadbagherpoor et al. "Exploring Airline Gate-Scheduling Optimization Using
    Quantum Computers", arXiv:2111.09472, url: https://arxiv.org/pdf/2111.09472

    Parameters:
    - G: List of gates.
    - P: List of planes.
    - V: Matrix (|P| x |P|) representing expected passengers between flights.
    - D: Matrix (|G| x |G|) representing walking distances between gates.
    - lambda1: Penalty weight for the constraint that each plane gets exactly one gate.
    - lambda2: Penalty weight for the constraint that each gate is assigned at most one plane.

    Returns:
    - Q: Dictionary representing the QUBO matrix.
    """

    num_planes = len(P)
    num_gates = len(G)
    Q = {}

    # Define binary variables x[i, k] where i is plane, k is gate
    def q_idx(i, k):
        """Index mapping (i, k) to a unique variable index."""
        return i * num_gates + k

    # Objective function: Minimize walking distance weighted by passenger flow
    for (i, j) in product(range(num_planes), repeat=2):
        if i != j:
            for (k, l) in product(range(num_gates), repeat=2):
                idx1, idx2 = q_idx(i, k), q_idx(j, l)
                Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + V[i][j] * D[k][l]

    # Constraint 1: Each plane is assigned exactly one gate
    for i in range(num_planes):
        for k in range(num_gates):
            idx = q_idx(i, k)
            Q[(idx, idx)] = Q.get((idx, idx), 0) - 2 * lambda1  # Linear term

        for k1, k2 in product(range(num_gates), repeat=2):
            if k1 != k2:
                idx1, idx2 = q_idx(i, k1), q_idx(i, k2)
                Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + 2 * lambda1  # Quadratic term

    # Constraint 2: Each gate is occupied by at most one plane
    for k in range(num_gates):
        for i in range(num_planes):
            idx = q_idx(i, k)
            Q[(idx, idx)] = Q.get((idx, idx), 0) + lambda2  # Linear term

        for i1, i2 in product(range(num_planes), repeat=2):
            if i1 != i2:
                idx1, idx2 = q_idx(i1, k), q_idx(i2, k)
                Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + 2 * lambda2  # Quadratic term

    return Q


In [4]:

def create_ising_coefficients(Q: Dict) -> Tuple[Dict, Dict]:
    """
    Converts a QUBO matrix into an Ising Hamiltonian.

    Parameters:
    - Q: Dictionary representing the QUBO matrix.

    Returns:
    - h: Dictionary of linear biases.
    - J: Dictionary of quadratic couplings.
    """
    h = {}
    J = {}

    for (i, j), value in Q.items():
        if i == j:
            h[i] = h.get(i, 0) + value  # Linear terms
        else:
            J[(i, j)] = J.get((i, j), 0) + value / 4  # Quadratic terms scaled for Ising

            # Adjust linear terms due to transformation x = (1 + s)/2
            h[i] = h.get(i, 0) + value / 4
            h[j] = h.get(j, 0) + value / 4

    return h, J


In [6]:

# Example Usage

# Example setup
G = ["G1", "G2", "G3"]  # 3 gates
P = ["P1", "P2"]        # 2 planes
V = np.array([[0, 10],  # Passenger flows
                [10, 0]])
D = np.array([[0, 5, 10],  # Walking distances
                [5, 0, 3],
                [10, 3, 0]])

# Create QUBO matrix
Q = create_qubo_matrix(G, P, V, D)

# Convert to Ising form
h, J = create_ising_coefficients(Q)

# Print results
print("QUBO Matrix (Dictionary Format):")
for key, value in Q.items():
    print(f"Q[{key}] = {value}")

print("\nIsing Hamiltonian Coefficients:")
print("h:", h)
print("J:", J)


QUBO Matrix (Dictionary Format):
Q[(0, 3)] = 20
Q[(0, 4)] = 50
Q[(0, 5)] = 100
Q[(1, 3)] = 50
Q[(1, 4)] = 20
Q[(1, 5)] = 30
Q[(2, 3)] = 100
Q[(2, 4)] = 30
Q[(2, 5)] = 20
Q[(3, 0)] = 20
Q[(3, 1)] = 50
Q[(3, 2)] = 100
Q[(4, 0)] = 50
Q[(4, 1)] = 20
Q[(4, 2)] = 30
Q[(5, 0)] = 100
Q[(5, 1)] = 30
Q[(5, 2)] = 20
Q[(0, 0)] = -10
Q[(1, 1)] = -10
Q[(2, 2)] = -10
Q[(0, 1)] = 20
Q[(0, 2)] = 20
Q[(1, 0)] = 20
Q[(1, 2)] = 20
Q[(2, 0)] = 20
Q[(2, 1)] = 20
Q[(3, 3)] = -10
Q[(4, 4)] = -10
Q[(5, 5)] = -10
Q[(3, 4)] = 20
Q[(3, 5)] = 20
Q[(4, 3)] = 20
Q[(4, 5)] = 20
Q[(5, 3)] = 20
Q[(5, 4)] = 20

Ising Hamiltonian Coefficients:
h: {0: np.float64(95.0), 3: np.float64(95.0), 4: np.float64(60.0), 5: np.float64(85.0), 1: np.float64(60.0), 2: np.float64(85.0)}
J: {(0, 3): np.float64(5.0), (0, 4): np.float64(12.5), (0, 5): np.float64(25.0), (1, 3): np.float64(12.5), (1, 4): np.float64(5.0), (1, 5): np.float64(7.5), (2, 3): np.float64(25.0), (2, 4): np.float64(7.5), (2, 5): np.float64(5.0), (3, 0): np.float64(5.